In [3]:
import ee
import xee
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
import xee
import cartopy.feature
import xarray as xr


# Initialize the Earth Engine module
ee.Initialize()

def fetch_and_process_era5(start_date, end_date, country_name='United Kingdom'):
    # Load the country boundaries
    countries = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')
    country = countries.filter(ee.Filter.eq('country_na', country_name))

    # Define the geographic domain by the specific country boundary
    area = country.geometry()

    # Load ERA5 daily dataset and clip it to the country boundary
    era5_daily = ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR') \
        .filterDate(start_date, end_date) \
        .map(lambda image: image.clip(area))
    
    # Define the conditions for skiable days
    wind_speed_threshold_mps = 50 / 2.23694  # Convert mph to m/s
    snow_depth_threshold_m = 0.25

    # Map function to check both conditions
    def check_conditions(image):
        # Calculate resultant wind speed
        u_wind = image.select('u_component_of_wind_10m')
        v_wind = image.select('v_component_of_wind_10m')
        resultant_wind_speed = u_wind.pow(2).add(v_wind.pow(2)).sqrt()

        # Check wind and snow conditions
        suitable_wind = resultant_wind_speed.lt(wind_speed_threshold_mps)
        suitable_snow = image.select('snow_depth').gt(snow_depth_threshold_m)
        
        # Return image with combined conditions
        return suitable_wind.And(suitable_snow).rename('skiable_day')

    # Apply the map function
    skiable_days = era5_daily.map(check_conditions)
    
    # Sum the skiable days
    total_skiable_days = skiable_days.reduce(ee.Reducer.sum()).rename('total_skiable_days')
    total_skiable_days = total_skiable_days.select('total_skiable_days')
    
    # Convert to xarray Dataset
    ds = xr.open_dataset(total_skiable_days, engine='ee')#, crs='EPSG:4326', scale=0.25)
    
    return ds

def plot_skiable_days_xarray(ds):
    # Plot using Matplotlib and Cartopy
    fig, ax = plt.subplots(figsize=(10, 12), subplot_kw={'projection': ccrs.PlateCarree()})
    ax.set_extent([-10, 2, 50, 60], crs=ccrs.PlateCarree())
    
    ax.add_feature(cartopy.feature.LAND, edgecolor='black')
    ax.add_feature(cartopy.feature.OCEAN)
    ax.add_feature(cartopy.feature.RIVERS)
    ax.add_feature(cartopy.feature.LAKES, alpha=0.5)
    ax.coastlines(resolution='10m')
    
    # Plot the data
    im = ax.imshow(ds.skiable_day.values, extent=[-10, 2, 50, 60], transform=ccrs.PlateCarree(), cmap='viridis', origin='upper')
    cbar = fig.colorbar(im, ax=ax, orientation='vertical', fraction=0.036, pad=0.04)
    cbar.set_label('Total Skiable Days')
    
    plt.title('Skiable Days in the UK (1950-1955)')
    plt.show()

# Define parameters
start_date = '1950-01-01'
end_date = '1955-12-31'

# Fetch and process data
ds = fetch_and_process_era5(start_date, end_date, 'United Kingdom')

# Plot the results
plot_skiable_days_xarray(ds)


EEException: ImageCollection.load: ImageCollection asset 'ee.Image({  "functionInvocationValue": {    "functionName": "Image.select",    "arguments": {      "bandSelectors": {        "constantValue": [          "total_skiable_days"        ]      },      "input": {        "functionInvocationValue": {          "functionName": "Image.rename",          "arguments": {            "input": {              "functionInvocationValue": {                "functionName": "ImageCollection.reduce",                "arguments": {                  "collection": {                    "functionInvocationValue": {                      "functionName": "Collection.map",                      "arguments": {                        "baseAlgorithm": {                          "functionDefinitionValue": {                            "argumentNames": [                              "_MAPPING_VAR_0_0"                            ],                            "body": {                              "functionInvocationValue"